# Program memory: dynamic memory allocation
_COSC 208, Introduction to Computer Systems, Fall 2025_

## Laws of pointers & memory allocation
1. A pointer variable holds a memory address
2. An array variable is a constant pointer to the 0th element

## Pointers as return values

* _What happens?_

In [3]:
#include <stdio.h>
int *one() {
    int x = 1;
    int *p = &x;
    return p;
}
int main() {
    int *q = one();
    printf("%d\n", *q);
}

1


* `q` points to a variable that no longer exists!
* _So, how can I return a pointer from a function?_ — dynamically allocate memory on the heap!

## Program memory
* Memory layout
    ```
    ┏━━━━━━━━━━━━━━━━━━┓ 0
    ┃ Operating system ┃
    ┣━━━━━━━━━━━━━━━━━━┫
    ┃       Code       ┃ <- Function instructions
    ┣━━━━━━━━━━━━━━━━━━┫
    ┃       Data       ┃ <- Global variables
    ┣━━━━━━━━━━━━━━━━━━┫
    ┃       Heap       ┃ <- Dyanmically allocated memory
    ┃         ▼        ┃    Grows toward higher memory addresses
    ┣━━━━━━━━━━━━━━━━━━┫    
    ┃                  ┃
    ┣━━━━━━━━━━━━━━━━━━┫ 
    ┃         ▲        ┃    Grows toward lower memory addresses
    ┃       Stack      ┃ <- Local variables and parameters
    ┗━━━━━━━━━━━━━━━━━━┛ Max address
    ```
    * Stack consists of stack frames – add a frame when a function is called, remove a frame when a function returns
* Variable storage
    * Local variables and parameters and stored on the stack --- in the frame for the function in which they are declared
    * Global variables are stored in the data section
* Memory allocation
    * Code and data — automatically allocated with a program starts
    * Stack — automatically allocated when a function is called; automatically deallocated when a function returns
    * Heap memory — explicitly allocated and freed by a program

## malloc

* `void* malloc(unsigned int size)`

**Laws of pointers & memory allocation 3: Memory allocated on the heap persists until explicitly freed**

* When to malloc?
    * When the amount of space required is not known until runtime
    * When a value must remain in memory even after returning from a function
* How much to malloc?
    * Use `sizeof` and a type: e.g., `sizeof(int)`
    * How much to malloc for an array? — multiply sizeof(type) by number of elements in array

<div style="height:5em;"></div>

Q2: _Write a function called `range` that behaves similar to the `range` function in Python. Your function should take an unsigned integer (`length`) as a parameter, and return a dynamically allocated array with `length` unsigned integers. The array should be populated with the values 0 through `length-1`._

In [10]:
#include <stdlib.h>
unsigned int *range(unsigned int length) {
    unsigned int *nums = malloc(sizeof(unsigned int) * length);
    for (int i = 0; i < length; i++) {
        nums[i] = i;
    }
    return nums;
}
// Testing
#include <assert.h>
int main() {
    unsigned int *result = range(3);
    assert(result[0] == 0 && result[1] == 1 && result[2] == 2);
}

<div style="height:20em;"></div>

Q3: _Assume you wanted to write a function that creates a copy of a string. What is wrong with each of the following attempts at writing such a function?_

In [ ]:
#include <string.h>
char *copy1(char strA[]) {
    char strB[strlen(strA) + 1];
    strcpy(strB, strA);
    return strB;
}

    You cannot return a locally-declared array

<p style="height:2em;"></p>

In [ ]:
#include <string.h>
#include <stdlib.h>
char copy2(char strA[]) {
    char *strB = malloc(sizeof(char) * (strlen(strA) + 1));
    strcpy(strB, strA);
    return *strB;
}

    Returns the first character in the copy, instead of an array of characters

<p style="height:2em;"></p>

In [ ]:
#include <string.h>
#include <stdlib.h>
char *copy3(char strA[]) {
    char *strB = malloc(sizeof(char *));
    strcpy(strB, strA);
    return strB;
}

    Allocates space for a pointer, not space for the number of characters in strA

<p style="height:2em;"></p>

Q4: _Write a function called `generate_password` that takes an unsigned integer (`length`) as a parameter, and returns a dynamically allocated array of with `length` randomly selected characters (e.g., uppercase letters, lowercase letters, digits, symbols). Your function should use the `rand()` function from the C standard library, which returns a pseudo-random integer in the range 0 to `RAND_MAX`._

In [13]:
#include <stdlib.h>
char *generate_password(unsigned int length) {
    char *password = malloc(sizeof(char) * (length + 1));
    for (int i = 0; i < length; i++) {
        password[i] = (rand() % ('~' - '!')) + '!';
    }
    password[length] = '\0';
    return password;
}
// Testing
#include <stdio.h>
int main() {
    printf("%s\n", generate_password(8));
}

d:8#U2`t


<div style="height:20em;"></div>

## 2D arrays

Assume we wanted to represent a Tic Tac Toe board
```
+---+---+---+
| X |   |   |
+---+---+---+
|   | X |   |
+---+---+---+
|   | O |   |
+---+---+---+
```

* _How would we create an empty board?_ – need to create a 2D array
* _How would we represent one row of the board?_ – as an array of characters

In [ ]:
// First attempt
#include <stdlib.h>
#define BOARD_SIZE 3
??? create_board() {
    char *top_row = malloc(sizeof(char) * BOARD_SIZE);
    char *middle_row = malloc(sizeof(char) * BOARD_SIZE);
    char *bottom_row = malloc(sizeof(char) * BOARD_SIZE);
    return ???; // How do we return three arrays?
}

* Create an array-of-arrays
* _An array is a..._ – pointer
* _What does an array-of-arrays store?_ – pointers
* _What is an array-of-arrays?_ – a pointer to the zeroth element that is a pointer to the zeroth element

In [ ]:
// Second attempt
#include <stdlib.h>
#define BOARD_SIZE 3
char **create_board() {
    char *top_row = malloc(sizeof(char) * BOARD_SIZE);
    char *middle_row = malloc(sizeof(char) * BOARD_SIZE);
    char *bottom_row = malloc(sizeof(char) * BOARD_SIZE);
    char **board = malloc(sizeof(char *) * BOARD_SIZE);
    board[0] = top_row;
    board[1] = middle_row;
    board[2] = bottom_row;
    return board;
}

In [ ]:
// Final version
#include <stdlib.h>
#define BOARD_SIZE 3
char **create_board() {
    char **board = malloc(sizeof(char *) * BOARD_SIZE);
    for (int r = 0; r < BOARD_SIZE; r++) {
        board[r] = malloc(sizeof(char) * BOARD_SIZE);
        for (int c = 0; c < BOARD_SIZE; c++) {
            board[r][c] = ' ';
        }
    }
    return board;
}

int main() {
    char **ttt = create_board();
    ttt[1][1] = 'X';
    
    for (int r = 0; r < BOARD_SIZE; r++) {
        free(ttt[r]);
    }
    free(ttt);
}

<div style="height:15em;"></div>

Q5: _Write a `print_board` function that prints the current state of a tic tac toe board._

In [2]:
#include <stdlib.h>
#include <stdio.h>
#define BOARD_SIZE 3
char **create_board() {
    char **board = malloc(sizeof(char *) * BOARD_SIZE);
    for (int r = 0; r < BOARD_SIZE; r++) {
        board[r] = malloc(sizeof(char) * BOARD_SIZE);
        for (int c = 0; c < BOARD_SIZE; c++) {
            board[r][c] = ' ';
        }
    }
    return board;
}
void print_board(char **board) {
    for (int r = 0; r < BOARD_SIZE; r++) {
        if (r != 0) {
            printf("---+---+---\n");
        }
        printf(" %c | %c | %c \n", board[r][0], board[r][1], board[r][2]);
    }
}
int main() {
    char **ttt = create_board();
    ttt[0][0] = 'X';
    ttt[1][0] = 'O';
    print_board(ttt);
}

 X |   |   
---+---+---
 O |   |   
---+---+---
   |   |   


<p style="height:20em;"></p>

Q6: _Write a function called `lengths` that takes an array of strings and the number of elements in the array and returns an array of integers containing the length of each string._

In [17]:
#include <stdlib.h>
#include <string.h>
int *lengths(char *strs[], int count) {
    int *lens = malloc(sizeof(int) * count);
    for (int i = 0; i < count; i++) {
        lens[i] = strlen(strs[i]);
    }
    return lens;
}
// Testing
#include <assert.h>
int main() {
    char *strs[] = {"abc", "12345", "do re mi"};
    int *lens = lengths(strs, 3);
    assert(lens[0] == 3 && lens[1] == 5 && lens[2] == 8);
}

<div style="height:20em;"></div>

Q7: _Write a `get_winner` function that takes a tic tac toe board and returns `X` if `X`s won, `O` if `O`s won, `T` if there is a tie, or `?` if there is not yet a winner or tie._

In [1]:
#define BOARD_SIZE 3
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
char **create_board() {
    char **board = malloc(sizeof(char *) * BOARD_SIZE);
    for (int r = 0; r < BOARD_SIZE; r++) {
        board[r] = malloc(sizeof(char) * BOARD_SIZE);
        for (int c = 0; c < BOARD_SIZE; c++) {
            board[r][c] = ' ';
        }
    }
    return board;
}
void print_board(char **board) {
    for (int r = 0; r < BOARD_SIZE; r++) {
        if (r != 0) {
            printf("---+---+---\n");
        }
        printf(" %c | %c | %c \n", board[r][0], board[r][1], board[r][2]);
    }
}
char get_winner(char **board) {
    // Check rows for win
    for (int r = 0; r < BOARD_SIZE; r++) {
        if (board[r][0] != ' ' && board[r][0] == board[r][1] && board[r][0] == board[r][2]) {
            return board[r][0];
        }
    }
    // Check cols for win
    for (int c = 0; c < BOARD_SIZE; c++) {
        if (board[0][c] != ' ' && board[0][c] == board[1][c] && board[0][c] == board[2][c]) {
            return board[0][c];
        }
    }
    // Check diagonals for win
    if (board[0][0] != ' ' && board[0][0] == board[1][1] && board[0][0] == board[2][2]) {
        return board[0][0];
    }
    if (board[0][2] != ' ' && board[0][2] == board[1][1] && board[0][2] == board[2][0]) {
        return board[0][2];
    }
    // Check diagonals for unknown state
    for (int r = 0; r < BOARD_SIZE; r++) {
        for (int c = 0; c < BOARD_SIZE; c++) {
            if (board[r][c] == ' ') {
                return '?';
            }
        }
    }
    return 'T';
}
int main() {
    char **ttt = create_board();
    ttt[0][0] = 'X';
    ttt[0][1] = 'X';
    print_board(ttt);
    assert(get_winner(ttt) == '?');
    ttt[0][2] = 'X';
    ttt[2][0] = 'O';
    ttt[2][2] = 'O';
    print_board(ttt);
    assert(get_winner(ttt) == 'X');
    ttt[0][0] = ' ';
    ttt[1][1] = 'X';
    ttt[2][1] = 'X';
    print_board(ttt);
    assert(get_winner(ttt) == 'X');
        ttt[0][0] = ' ';
    ttt[2][0] = 'X';
    ttt[2][1] = ' ';
    print_board(ttt);
    assert(get_winner(ttt) == 'X');
}

<p style="height:18em;"></p>

## free

* `void free(void *block)`
    * Block must be the same as the value returned by malloc (i.e., a pointer to the beginning of the allocated memory region)
    * Entire memory region is freed; cannot free part of a memory region
* When to free? — when a value stored on the heap is no longer needed
    * Free memory regions as soon as you are done
    * Every call to malloc should have a corresponding call to free
    * Do not read/write the memory location after it has been freed!

Q8: _What memory deallocation mistake has been made in each of the following programs?_

In [ ]:
#include <stdlib.h>
int main() {
    int *ptrA = malloc(sizeof(int) * 3);
    int *ptrB = ptrA;
    free(ptrA);
    free(ptrB);
}

_Double free_

<p style="height:2em;"></p>

In [ ]:
#include <stdlib.h>
int main() {
    int *ptr = malloc(sizeof(int) * 3);
    ptr++;
    free(ptr);
}

_Not freeing from beginning of allocated region_

<p style="height:2em;"></p>

In [1]:
#include <stdlib.h>
int main() {
    int *ptrA = malloc(sizeof(int) * 3);
    int *ptrB = ptrA;
    ptrA[0] = 0;
    ptrB[1] = 1;
    free(ptrA);
    ptrB[2] = 2;
}

_Use after free_

<p style="height:2em;"></p>

Q9: _Write a `destroy_board` function that deallocates a tic tac toe board._

In [3]:
#define BOARD_SIZE 3
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
char **create_board() {
    char **board = malloc(sizeof(char *) * BOARD_SIZE);
    for (int r = 0; r < BOARD_SIZE; r++) {
        board[r] = malloc(sizeof(char) * BOARD_SIZE);
        for (int c = 0; c < BOARD_SIZE; c++) {
            board[r][c] = ' ';
        }
    }
    return board;
}
void destroy_board(char **board) {
    for (int r = 0; r < BOARD_SIZE; r++) {
        free(board[r]);
    }
    free(board);
}
int main() {
    char **ttt = create_board();
    destroy_board(ttt);
}

<p style="height:18em;"></p>